In [2]:
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
import seaborn as sns

FILES_FOLDER = {
    "data_for_data_challenge": [
        "data_for_data_challenge/building_data.feather",
        "data_for_data_challenge/meter_data.feather",
        "data_for_data_challenge/weather_data.feather",  # The dataset test will be released one hour before the end of the Data Challenge
    ]
}


def load_data(folder, data_dir, dict_files=FILES_FOLDER):
    files = dict_files[folder]
    dataframes = []
    print(":: Start loading data")
    for name_file in tqdm(files):
        dataframe = pd.read_feather(os.path.join(data_dir, name_file))
        dataframes.append(dataframe)
    return dataframes


building, meters, weather = load_data("data_for_data_challenge", "")

meters["naive_pred"] = meters["meter_reading"].mean()

print(
    "Naive performance on the training set :",
    mean_absolute_percentage_error(meters["meter_reading"], meters["naive_pred"]),
)

:: Start loading data


100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 19.25it/s]

Naive performance on the training set : 1.497798e+18


In [3]:
building.head()
print(building.shape)

(466, 9)


In [4]:
print(len(building[building['primary_use']=="Other"]))

10


On voit qu'il n'y a que peu de buildings dont le primary_use est non spécifié (other). Comme il semble que le rôle du bâtiment joue dans la consommation (un parking va par exemple chauffer nettement moins qu'une résidence), on peut les enlever.

In [25]:
new_building = building[building['primary_use']!="Other"] #pour l'instant je le crée mais il servira à rien
new_building.head() #parce que on a pas décidé si on voulait les supprimer
print(new_building.shape)

,index,site_id,building_id,primary_use,sub_primary_use,square_feet,lat,lng,year_built
1,1300,0,56,Lodging/residential,Residence Hall\t,60088.0,28.52,-81.400002,2013.0
2,2645,0,38,Office,Office,12769.0,28.52,-81.400002,2013.0
3,3939,0,71,Parking,Parking Garage,387638.0,28.52,-81.400002,1998.0
4,5294,0,46,Retail,Retail,9045.0,28.52,-81.400002,2016.0
5,6596,0,64,Lodging/residential,Residence Hall,53130.0,28.52,-81.400002,2001.0


In [34]:
new_building['site_id'].unique() #ouf, on a pas un site avec que des other en primary_use

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [27]:
new_building['lng'].isna().sum() #Idem pour lat, c'est quand même beaucoup...

62

In [28]:
no_NaN_lat=new_building.dropna(subset = ['lng'], inplace=False)

In [29]:
no_NaN_lat['site_id'].unique() #là on a quand même un peu déconné, les sites 8,10 et 14 disparaissent complètement 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 11, 12, 13, 15])

Comme enlever les rows sans localisation nous supprime quand même trois sites entiers, on va garder ces rows mais il faut garder en tête qu'on a un problème de NaN, le plus simple serait soit de supprimer le critère localisation soit de supprimer les sites 8, 10 et 14.

In [30]:
new_building.to_csv("clean_data/building.csv")